# IRAF-like source detection with simulated images

## Overview

IRAF provided a couple of ways, `starfind` and `daofind`, to detect *stellar* sources in an image. [photutils]() provides similar functionality via the [`DAOStarFinder`]() and [`IRAFStarFinder`]() objects

This notebook will focus on [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) to emphasize that these options work well for detecting *stars* or other objects with a stellar profile. They do not work well for more extended objects.

The next couple of notebooks will:

+ apply these techniques to the Hubble Extreme Deep Field, to illustrate the differences between [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) and [`IRAFStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.IRAFStarFinder.html#photutils.detection.IRAFStarFinder), and to introduce another method of source detection,
+ apply the techniques to an image of stars from a ground-based telescope, and
+ illustrate other source detection techniques that work well for extended sources

## Simulated image of galaxies and stars

In the first part of this notebook we consider an image that includes 100 sources, all with Gaussian profiles, some of which are fairly elongated. We used this image in the previous section about removing the background prior to source detection.

As usual, we begin with some imports.

In [ ]:
from astropy.stats import sigma_clipped_stats, gaussian_sigma_to_fwhm
from astropy.table import QTable
from astropy.visualization import simple_norm, SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

import matplotlib.pyplot as plt
import numpy as np

from photutils.aperture import CircularAperture, EllipticalAperture
from photutils.datasets import make_100gaussians_image, make_gaussian_sources_image
from photutils.detection import find_peaks, DAOStarFinder

plt.style.use('../photutils_notebook_style.mplstyle')

To begin, let's create and look the the image. We create an image normalization here that we will use for displaying the image throughout the notebook.

In [ ]:
data = make_100gaussians_image()
norm = ImageNormalize(stretch=SqrtStretch())
plt.figure(figsize=(10, 5))
plt.imshow(data, cmap='Greys_r', origin='lower', norm=norm,
           interpolation='nearest')
plt.grid()

Note that some of these object are star-like -- the ones that are darkest are close to circular, for example. Others are very elongated and there are a variety of brightnesses among the objects.

### Estimate and subtract background 

The background must be subtracted from the image before the sources are detected. For simplicity we estimate the sigma-clipped mean, median and standard deviation of the pixels in the image. As discussed in the section on [background removal for source detection](), the sigma-clipped median gives a reasonable estimate of the background in many cases.

In [ ]:
mean, med, std = sigma_clipped_stats(data, sigma=3.0, maxiters=5)
data_subtracted = data - med

### Detect sources

The source detection itself is a couple of lines of code:

In [ ]:
detect_fwhm = 5.0
daofind = DAOStarFinder(fwhm=detect_fwhm, threshold=5.*std) 
sources = daofind(data_subtracted)

There are a variety of options you can select, detailed in the documentation for [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder). We only use two for now, specifying the typical full-width half-max (FWHM) of a source, and the threshold above which to conider something a source.

A summary of the sources detected shows that [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) detect 41 sources, substantially fewer than the 100 that are in the image.

In [ ]:
# Format the columns to only display two decimal points 
for col in sources.colnames:  
    if col not in ('id', 'npix'):
        sources[col].info.format = '%.2f'  # for consistent table output
sources.pprint(max_width=76)  


A plot showing the location of all of the detected sources helps illustrate why some of sources were missed. To display where sources were detected, we create a a set of [`photutils`](https://photutils.readthedocs.io/en/stable/) circular apertures and use its plot method to add a circle over each detected source.

In [ ]:
# CircularAperture expects a particular format for input positions...
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=detect_fwhm / 2)

plt.figure(figsize=(20, 10))
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1, alpha=0.5);
plt.title("Sources detected by DAOStarFinder")

It looks like the sources that were detected are those that were not too faint and not too elongaetd. 

Let's look at the properties of the sources in this image, and explore some of the settings in [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) to see how you could tune its parameters to detect a different subset of the sources.

### Sources in the image

The sources in this image are generated using the code below, which was copied from the [photutils source code here](https://photutils.readthedocs.io/en/stable/_modules/photutils/datasets/make.html#make_100gaussians_image).

The sources have a range of orientations, ellipticities, and widths. All of the sources are 2D Gaussians, with a FWHM ranging from roughly 3 to 15 pixels in each direction. The ratio of minor to major axis of the sources varies from 0.2 to 1.0.

In [ ]:
n_sources = 100
flux_range = [500, 1000]
xmean_range = [0, 500]
ymean_range = [0, 300]
xstddev_range = [1, 5]
ystddev_range = [1, 5]
params = {'flux': flux_range,
          'x_mean': xmean_range,
          'y_mean': ymean_range,
          'x_stddev': xstddev_range,
          'y_stddev': ystddev_range,
          'theta': [0, 2 * np.pi]}

rng = np.random.RandomState(12345)
inp_sources = QTable()
for param_name, (lower, upper) in params.items():
    # Generate a column for every item in param_ranges, even if it
    # is not in the model (e.g., flux).  However, such columns will
    # be ignored when rendering the image.
    inp_sources[param_name] = rng.uniform(lower, upper, n_sources)
xstd = inp_sources['x_stddev']
ystd = inp_sources['y_stddev']
inp_sources['amplitude'] = inp_sources['flux'] / (2.0 * np.pi * xstd * ystd)

Let's look at the image again, with a marker around each input source. There is quite a bit of information packed into those markers:

+ the shape of the marker matches the minor-to-major axis ratio
+ the major axis of the marker is the FWHM of the major axis
+ the thickness of the marker edge indicates the amplitude of the source.


Next, we overlay the sources that were detected.

In [ ]:
plt.figure(figsize=(20, 10))

# plot the image
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')

def plot_input_sources(ax=None, label_text=""):
    if ax is None:
        ax = plt.gca()
        
    amplitudes = sorted(inp_sources['amplitude'])
    labeled = False
    for source in inp_sources:
        minor = min(source['x_stddev'], source['y_stddev'])
        major = max(source['x_stddev'], source['y_stddev'])
        ratio = minor / major
    
        major_fwhm = major * gaussian_sigma_to_fwhm
    
        # ap = EllipticalAperture((source['x_mean'], source['y_mean']), major, minor, theta=source['theta'])
        ap = EllipticalAperture((source['x_mean'], source['y_mean']), source['x_stddev'], source['y_stddev'], theta=source['theta'])
        
        # This should give linewidths of 1, 2, or 3 given the input fluxes
        line_width = source['flux'] // 250 - 1
    
        if source['amplitude'] > amplitudes[2 * len(amplitudes) // 3]:
            line_width = 3
        elif source['amplitude'] < amplitudes[len(amplitudes) // 3]:
            line_width = 1
        else:
            line_width = 2
        # Add a label to only the first aperture plotted, otherwise each aperture will be listed individually 
        # in the legend.
        label_kw = {"label": label_text} if not labeled else {}
        labeled = True
        ap.plot(color='green', lw=line_width, alpha=0.5, ax=ax, **label_kw)

plot_input_sources()
plt.title("All sources in the image");

In [ ]:
plt.figure(figsize=(20, 10))
# plot the image
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1, alpha=1);
for idx, ap in enumerate(apertures):
    label_offset = 3
    plt.annotate(str(idx), (ap.positions[0] + label_offset, ap.positions[1] + label_offset))
plot_input_sources()
plt.title("All sources (green) and detected sources (blue)\nlabeled by detected source number");


From this plot it seems that the sources detected are those that are:

+ about the same size as the FWHM we input to the source finder (5 pixels in this case) -- see sources labeled 5 and 22, for example.
+ reasonably round *or* reasonably concentrated -- see source 33 for an example. That source is very elongated but also very compact.

In some cases [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) finds *two* sources where there is really one. Sources 5 and 6 are a good example of this.

### An exploration of some parameters of [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) 

We can test some of the assertions in the previous paragraph. A couple of examples of that are to try

1. A lower threshold in detection to try to detect the fainter sources in this test image. Below we will try a threshold of 3 times the standard deviation of the background instead of 5 times.
2. A ratio smaller than 1 to detect the more elongated sources. The ratio argument is the ratio of the semiminor to semimajor axis. Below we will try a ratio of 0.33.

The code below begins with a function to do the plotting for this so that we can reuse that code for each plot.

In [ ]:
def detect_sources_and_plot_apertures(data, fwhm=0, threshold_factor=5, ratio=1, ax=None, color='blue', label_text=""):
    if ax is None:
        ax = plt.gca()
        
    daofind = DAOStarFinder(fwhm=detect_fwhm, threshold=threshold_factor * std, ratio=ratio) 
    sources = daofind(data_subtracted)
    positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
    apertures = CircularAperture(positions, r=detect_fwhm / 2)

    # Plot the first aperture with a label
    apertures[0].plot(color=color, lw=1, alpha=0.5, ax=ax, label=label_text)
    # Plot the rest without a label, otherwise each aperture gets its own label.
    apertures[1:].plot(color=color, lw=1, alpha=0.5, ax=ax)

fig, axes = plt.subplot_mosaic([['case 1', 'case 2']], figsize=[20, 10], sharey=True, tight_layout=True)

for ax in axes.values():
    ax.imshow(data, cmap='Greys', origin='lower', norm=norm, interpolation='nearest')
    plot_input_sources(ax=ax, label_text="input source")

# The lower threshold case
thresh = 3
detect_sources_and_plot_apertures(data_subtracted, fwhm=detect_fwhm, threshold_factor=5, ratio=1, ax=axes['case 1'], color='blue', label_text="original parameters")
detect_sources_and_plot_apertures(data_subtracted, fwhm=detect_fwhm, threshold_factor=thresh, ratio=1, ax=axes['case 1'], color='red', label_text="lower threshold")
axes['case 1'].legend(loc="upper left")
axes['case 1'].set_title(f"Source detection with threshold {thresh:.1f}")


# The small ratio case
ratio = 0.3
detect_sources_and_plot_apertures(data_subtracted, fwhm=detect_fwhm, threshold_factor=5, ratio=1, ax=axes['case 2'], color='blue', label_text="original parameters")
detect_sources_and_plot_apertures(data_subtracted, fwhm=detect_fwhm, ratio=ratio, ax=axes['case 2'], color="red", label_text="small ratio")
axes['case 2'].legend(loc="upper left")
axes['case 2'].set_title(f"Source detection with ratio {ratio:.2f}");

The plot on the left shows that lowering the detetion threshold does find more sources, but it also includes erroneous detections. The plot on right demonstrates that we can detect some (but not all) of the highly elongated objects by changing the `ratio` parameter in [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder). The reason that we don't pick up more of the elongated sources is that the orientation of the source matters too. The default value for the orientation is zero, i.e. aligned with the horizontal axis.

## Summary

The takeway from this section should be that the IRAF-like methods implemented in photutils are good at finding star-like sources. As we will see in the next couple of sections, the methods work very well for detecting stars or galaxies that are small enough to look like stars. After that we will look at a couple of methods, local peak detection and image segmentation, that are better for detecting extended sources.